In [1]:
import numpy as np
import glob, tqdm
import sys, dill
sys.path.append('../src')
from lib.utils import *

/data/kai/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sort_fn(filename):
    date_string = filename[-14:-4]
    datetime_object = datetime.datetime.strptime(date_string, "%Y-%m-%d")
    return datetime_object
def save_object(obj, filename):
    dill.dump(obj, file=open(filename, "wb"))

data_dir = "/data/allen/climate_data/"
splits = ["CAM5", "SPCAM5"]

for split in splits:
    x_paths = sorted(glob.glob(f"{data_dir}/{split}/inputs*"), key=sort_fn)[:365]
    y_paths = sorted(glob.glob(f"{data_dir}/{split}/outputs*"), key=sort_fn)[:365]

    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    for i, (x_f, y_f) in tqdm.tqdm(enumerate(zip(x_paths, y_paths)), total=len(y_paths)):
        x = np.load(x_f, mmap_mode='r')
        y = np.load(y_f, mmap_mode='r')
        x_scaler.fit(x)
        y_scaler.fit(y)

    save_object(x_scaler, f"x_{split}_minmax_scaler.dill")
    save_object(y_scaler, f"y_{split}_minmax_scaler.dill")

100%|██████████| 365/365 [01:48<00:00,  3.35it/s]


In [6]:
x_scaler = dill.load(open(f"x_SPCAM5_minmax_scaler.dill", 'rb'))
y_scaler = dill.load(open(f"y_SPCAM5_minmax_scaler.dill", 'rb'))

In [7]:
x_scaler.max

array([2.57015228e+02, 2.57642090e+02, 2.54332870e+02, 2.53782227e+02,
       2.52368683e+02, 2.47518005e+02, 2.40695084e+02, 2.39414856e+02,
       2.37699371e+02, 2.36804199e+02, 2.37789948e+02, 2.41294785e+02,
       2.43667007e+02, 2.43297791e+02, 2.47623642e+02, 2.55565735e+02,
       2.63735596e+02, 2.71335327e+02, 2.77857758e+02, 2.85017700e+02,
       2.94361694e+02, 3.02865021e+02, 3.09409851e+02, 3.15537903e+02,
       3.19438873e+02, 3.25349854e+02, 2.77338245e-06, 2.74451418e-06,
       3.21395123e-06, 1.16921483e-05, 1.33854410e-04, 7.74772343e-05,
       1.60562631e-05, 1.46545435e-05, 1.75437945e-05, 2.25672538e-05,
       5.30398065e-05, 1.44150996e-04, 3.76174314e-04, 8.89459858e-04,
       1.79885037e-03, 3.18541122e-03, 4.94139036e-03, 6.95395330e-03,
       9.00923740e-03, 1.14075588e-02, 1.43698975e-02, 1.73563585e-02,
       1.95446908e-02, 2.13226788e-02, 2.31636036e-02, 2.94124391e-02,
       0.00000000e+00, 0.00000000e+00, 1.02201204e-36, 4.29128419e-35,
      

In [8]:
x = np.load(x_paths[0])
x = x_scaler.transform(x)

In [9]:
x.min(), x.max()

(0.0, 1.0)

In [10]:
y = np.load(y_paths[2])
y = y_scaler.transform(y)

In [20]:
y.min(), y.max()

(0.0, 1.0)